In [1]:
import pandas as pd
import numpy as np
import shutil
import glob
import os
import datetime as dt
import multiprocessing as mp
import pyswmm
from pyswmm import Simulation, Links
import swmmio
import swmmio.utils.modify_model

# inputs

In [2]:
input_idf_table_file_path = './Assignment_3__Input_FE/Data_Centroid/BOM_IDF_Data/depths_-33.8774_151.093_all_design.csv'
input_temp_pattern_file_path = './Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_Increments.csv'
input_storm_stats_file_path = './Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_AllStats.csv'
output_dir = './Assignment_3__Output_FE__1st__centroid/'
original_inp_file_path = './Assignment_3__Input_FE/971007_SW5.INP'
simulation_starting_time = '2000/01/01 00:00:00'
stopping_time_after_precipitation_finish = {'days':0, 'hours':6, 'minutes':0, 'seconds':0}
timeseries_name = 'RainGauge'

simulation_starting_time = pd.to_datetime(simulation_starting_time)
stopping_time_after_precipitation_finish = pd.Timedelta(**stopping_time_after_precipitation_finish)

print(
    input_idf_table_file_path,
    input_temp_pattern_file_path,
    input_storm_stats_file_path,
    original_inp_file_path,
    simulation_starting_time,
    stopping_time_after_precipitation_finish,
    timeseries_name,
    sep='\n'
    )

./Assignment_3__Input_FE/Data_Centroid/BOM_IDF_Data/depths_-33.8774_151.093_all_design.csv
./Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_Increments.csv
./Assignment_3__Input_FE/Data_Centroid/ECsouth/ECsouth_AllStats.csv
./Assignment_3__Input_FE/971007_SW5.INP
2000-01-01 00:00:00
0 days 06:00:00
RainGauge


In [3]:
def create_output_dir(arg_output_dir):
    """create output directory if it does not exist

    arguments:
        [string] --> arg_output_dir = path of the output directory name
    """
    if not os.path.exists(arg_output_dir):
        os.makedirs(arg_output_dir)

In [4]:
create_output_dir(output_dir)
output_dir

'./Assignment_3__Output_FE__1st__centroid/'

In [5]:
inp_file_dir = '{}inp_file_dir/'.format(output_dir)
create_output_dir(inp_file_dir)
inp_file_dir

'./Assignment_3__Output_FE__1st__centroid/inp_file_dir/'

In [6]:
flow_rate_dir = '{}flow_rate_dir/'.format(output_dir)
create_output_dir(flow_rate_dir)
flow_rate_dir

'./Assignment_3__Output_FE__1st__centroid/flow_rate_dir/'

# Threads

In [7]:
n_core = mp.cpu_count() - 1
n_core

13

# setting up the inp file

In [8]:
inp_file_path = '{}inp_template.inp'.format(output_dir)
inp_file_path

'./Assignment_3__Output_FE__1st__centroid/inp_template.inp'

In [9]:
# copy the original inp file
shutil.copyfile(src=original_inp_file_path, dst=inp_file_path)

'./Assignment_3__Output_FE__1st__centroid/inp_template.inp'

In [10]:
options_inp_file = swmmio.utils.dataframes.dataframe_from_inp(inp_path=inp_file_path, section='[OPTIONS]')
options_inp_file.at['START_DATE', 'Value'] = simulation_starting_time.strftime(format='%m/%d/%Y')
options_inp_file.at['START_TIME', 'Value'] = simulation_starting_time.strftime(format='%H:%M:%S')
options_inp_file.at['REPORT_START_DATE', 'Value'] = simulation_starting_time.strftime(format='%m/%d/%Y')
options_inp_file.at['REPORT_START_TIME', 'Value'] = simulation_starting_time.strftime(format='%H:%M:%S')
options_inp_file.at['END_DATE', 'Value'] = (simulation_starting_time + stopping_time_after_precipitation_finish).strftime(format='%m/%d/%Y')
options_inp_file.at['END_TIME', 'Value'] = (simulation_starting_time + stopping_time_after_precipitation_finish).strftime(format='%H:%M:%S')
options_inp_file.at['REPORT_STEP', 'Value'] = str(pd.Timedelta(minutes=5)).split(sep=' ')[-1]
options_inp_file.at['WET_STEP', 'Value'] = str(pd.Timedelta(minutes=5)).split(sep=' ')[-1]
options_inp_file.at['DRY_STEP', 'Value'] = str(pd.Timedelta(minutes=5)).split(sep=' ')[-1]
options_inp_file.at['ROUTING_STEP', 'Value'] = str(pd.Timedelta(minutes=5)).split(sep=' ')[-1]
options_inp_file.at['THREADS', 'Value'] = n_core

options_inp_file

,Value
Key,
FLOW_UNITS,CMS
INFILTRATION,HORTON
FLOW_ROUTING,KINWAVE
LINK_OFFSETS,DEPTH
MIN_SLOPE,0
ALLOW_PONDING,NO
SKIP_STEADY_STATE,NO
START_DATE,01/01/2000
START_TIME,00:00:00


In [11]:
raingauges_inp_file = swmmio.utils.dataframes.dataframe_from_inp(inp_path=inp_file_path, section='[RAINGAGES]')
raingauges_inp_file = raingauges_inp_file.iloc[:1]
raingauges_inp_file.index = pd.Index(data=[timeseries_name], name='Name')
raingauges_inp_file.at[timeseries_name, 'TimeIntrvl'] = str(pd.Timedelta(minutes=5)).split(sep=' ')[-1].rsplit(sep=':', maxsplit=1)[0]
raingauges_inp_file.at[timeseries_name, 'DataSourceName'] = timeseries_name

raingauges_inp_file

,RainType,TimeIntrvl,SnowCatch,DataSource,DataSourceName
Name,,,,,
RainGauge,VOLUME,00:05,1.0,TIMESERIES,RainGauge


In [12]:
subcatchments_inp_file = swmmio.utils.dataframes.dataframe_from_inp(inp_path=inp_file_path, section='[SUBCATCHMENTS]')
subcatchments_inp_file.Raingage = timeseries_name

subcatchments_inp_file

,Raingage,Outlet,...,PercSlope,CurbLength
Name,,,,,
2101,RainGauge,101,...,3.50,0
2401,RainGauge,401,...,4.50,0
2102,RainGauge,102,...,4.80,0
2103,RainGauge,103,...,4.50,0
2104,RainGauge,104,...,4.00,0
...,...,...,...,...,...
2921,RainGauge,921,...,0.00,0
2922,RainGauge,922,...,0.00,0
2923,RainGauge,923,...,4.55,0


In [13]:
timeseries_inp_file = swmmio.utils.dataframes.dataframe_from_inp(inp_path=inp_file_path, section='[TIMESERIES]')
timeseries_inp_file.reset_index(inplace=True)
timeseries_inp_file.drop(index=timeseries_inp_file.index[1:], inplace=True)
timeseries_inp_file.at[0, 'Name'] = timeseries_name
timeseries_inp_file.set_index(keys='Name', inplace=True)
timeseries_inp_file.at[timeseries_name, 'Date'] = simulation_starting_time.strftime(format='%m/%d/%Y')
timeseries_inp_file.at[timeseries_name, 'Time'] = simulation_starting_time.strftime(format='%H:%M')
timeseries_inp_file.at[timeseries_name, 'Value'] = 0.0

timeseries_inp_file

,Date,Time,Value
Name,,,
RainGauge,01/01/2000,00:00,0.0


In [14]:
[swmmio.utils.modify_model.replace_inp_section(inp_path=inp_file_path, modified_section_header=ind1, new_data=ind2)
for ind1, ind2 in zip(
    ['[OPTIONS]', '[RAINGAGES]', '[SUBCATCHMENTS]', '[TIMESERIES]'], 
    [options_inp_file, raingauges_inp_file, subcatchments_inp_file, timeseries_inp_file]
    )]

# building idf table

In [15]:
idf_table = pd.read_csv(filepath_or_buffer=input_idf_table_file_path, skiprows=9)
frequency_label = idf_table.columns[2:].to_numpy()
idf_table.columns = idf_table.columns.to_series().apply(
    func=lambda arg: 
        '_'.join(arg.split(sep=' ')) 
        if arg[:8] == 'Duration' 
        else 'freq_' + '_'.join('_perc'.join('_'.join(arg.split(sep=' ')).split(sep='%')).split(sep='.'))
    )
frequency_tag = idf_table.columns[2:].to_numpy()
idf_table[['Duration', 'units']] = idf_table.apply(func=lambda arg: arg.Duration.split(sep=' '), axis=1, result_type='expand')
idf_table = idf_table[idf_table.columns[[0,-1] + list(range(1,len(idf_table.columns) - 1))]]
idf_table.Duration = idf_table.Duration.astype(float)
idf_table.to_csv(path_or_buf='{}table_idf_depths.csv'.format(output_dir), index=False)
idf_table.to_parquet(path='{}table_idf_depths.parquet'.format(output_dir))

idf_table

,Duration,units,...,freq_1_in_1000,freq_1_in_2000
0,1.0,min,...,7.17,7.76
1,2.0,min,...,11.30,12.20
2,3.0,min,...,15.80,17.10
3,4.0,min,...,20.10,21.80
4,5.0,min,...,24.00,26.10
5,10.0,min,...,39.00,42.20
6,15.0,min,...,48.70,52.80
7,20.0,min,...,55.70,60.40
8,25.0,min,...,61.20,66.30
9,30.0,min,...,65.60,71.10


# building temporal pattern table

In [16]:
temp_pattern_table = pd.read_csv(filepath_or_buffer=input_temp_pattern_file_path)
temp_pattern_table.rename(columns=lambda arg: arg.strip(), inplace=True)
# temp_pattern_table.rename(columns={'Duration':'Duration_in_min'}, inplace=True)
temp_pattern_table.columns = pd.Index(data=temp_pattern_table.columns[:5].to_list() + ['Increment_{:02}'.format(ind) for ind in range(len(temp_pattern_table.columns[5:]))])
temp_pattern_table.to_csv(path_or_buf='{}table_temp_patterns.csv'.format(output_dir), index=False)
temp_pattern_table.to_parquet(path='{}table_temp_patterns.parquet'.format(output_dir))

temp_pattern_table

,EventID,Duration,...,Increment_54,Increment_55
0,4380,10,...,NaN,NaN
1,4382,10,...,NaN,NaN
2,4384,10,...,NaN,NaN
3,4385,10,...,NaN,NaN
4,4386,10,...,NaN,NaN
...,...,...,...,...,...
715,2856,10080,...,0.02,0.00
716,5064,10080,...,0.40,0.11
717,5069,10080,...,0.12,3.09
718,5071,10080,...,0.00,0.15


In [17]:
temp_pattern_table.dtypes

EventID           int64
Duration          int64
TimeStep          int64
Region           object
AEP              object
                 ...   
Increment_51    float64
Increment_52    float64
Increment_53    float64
Increment_54    float64
Increment_55    float64
Length: 61, dtype: object

In [18]:
storm_stats_table = pd.read_csv(filepath_or_buffer=input_storm_stats_file_path)
storm_stats_table.rename(columns=lambda arg: arg.strip(), inplace=True)
for ind in [(' ','_'), ('(',''), (')',''), ('%','in_perc'), ('_No.','_Number'), ('_min','_in_min'), ('_mm','_in_mm')]:
    storm_stats_table.rename(columns=lambda arg: arg.replace(*ind), inplace=True)
# storm_stats_table.rename(columns={'Burst_Duration_in_min':'Duration_in_min'}, inplace=True)
storm_stats_table.dropna(axis=0, inplace=True)
storm_stats_table.Burst_Start_Date = pd.to_datetime(arg=storm_stats_table.Burst_Start_Date, format='%m/%d/%Y %H:%M')
storm_stats_table.Burst_End_Date = pd.to_datetime(arg=storm_stats_table.Burst_End_Date, format='%m/%d/%Y %H:%M')
storm_stats_table.Event_ID = storm_stats_table.Event_ID.astype(int)
storm_stats_table.Burst_Duration_in_min = storm_stats_table.Burst_Duration_in_min.astype(int)
storm_stats_table.Burst_Loading = storm_stats_table.Burst_Loading.astype(int)
storm_stats_table.DB_Event_Reference_Number = storm_stats_table.DB_Event_Reference_Number.astype(int)
storm_stats_table.DB_Pluviograph_Reference_Number = storm_stats_table.DB_Pluviograph_Reference_Number.astype(int)
storm_stats_table.to_csv(path_or_buf='{}table_storm_stats.csv'.format(output_dir), index=False)
storm_stats_table.to_parquet(path='{}table_storm_stats.parquet'.format(output_dir))

storm_stats_table

,Event_ID,Region,...,Lat,Long
0,4380,East Coast (South),...,-32.2375,150.6306
1,4382,East Coast (South),...,-32.6296,151.5919
2,4384,East Coast (South),...,-34.4869,150.4019
3,4385,East Coast (South),...,-33.7944,150.5083
4,4386,East Coast (South),...,-34.0333,150.2153
...,...,...,...,...,...
715,2856,East Coast (South),...,-27.4178,153.1142
716,5064,East Coast (South),...,-26.1831,152.6414
717,5069,East Coast (South),...,-24.9069,152.3230
718,5071,East Coast (South),...,-27.0258,152.5642


# building frequency classification table

In [19]:
ey_constants = np.array(object=[12,6,4,3,2,1,0.5,0.2])
ey_constants

array([12. ,  6. ,  4. ,  3. ,  2. ,  1. ,  0.5,  0.2])

In [20]:
aep_variables = np.multiply(np.add(1, np.negative(np.exp(np.negative(ey_constants)))),100)
aep_variables

array([99.99938558, 99.75212478, 98.16843611, 95.02129316, 86.46647168,
       63.21205588, 39.34693403, 18.12692469])

In [21]:
aep_constants = np.array(object=[50, 20, 10, 5, 2, 1, 0.5, 0.2, 0.1, 0.05, 0.02])
aep_constants

array([5.e+01, 2.e+01, 1.e+01, 5.e+00, 2.e+00, 1.e+00, 5.e-01, 2.e-01,
       1.e-01, 5.e-02, 2.e-02])

In [22]:
ey_variables = np.negative(np.log(np.add(1, np.negative(np.divide(aep_constants, 100)))))
ey_variables

array([6.93147181e-01, 2.23143551e-01, 1.05360516e-01, 5.12932944e-02,
       2.02027073e-02, 1.00503359e-02, 5.01254182e-03, 2.00200267e-03,
       1.00050033e-03, 5.00125042e-04, 2.00020003e-04])

In [23]:
ey = np.flip(m=np.sort(a=np.concatenate((ey_constants, ey_variables))))
ey

array([1.20000000e+01, 6.00000000e+00, 4.00000000e+00, 3.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 6.93147181e-01, 5.00000000e-01,
       2.23143551e-01, 2.00000000e-01, 1.05360516e-01, 5.12932944e-02,
       2.02027073e-02, 1.00503359e-02, 5.01254182e-03, 2.00200267e-03,
       1.00050033e-03, 5.00125042e-04, 2.00020003e-04])

In [24]:
aep_percentage = np.flip(m=np.sort(a=np.concatenate((aep_constants, aep_variables))))
aep_percentage

array([9.99993856e+01, 9.97521248e+01, 9.81684361e+01, 9.50212932e+01,
       8.64664717e+01, 6.32120559e+01, 5.00000000e+01, 3.93469340e+01,
       2.00000000e+01, 1.81269247e+01, 1.00000000e+01, 5.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 5.00000000e-01, 2.00000000e-01,
       1.00000000e-01, 5.00000000e-02, 2.00000000e-02])

In [25]:
aep_1_in_x = np.divide(100, aep_percentage)
aep_1_in_x

array([1.00000614e+00, 1.00248491e+00, 1.01865736e+00, 1.05239570e+00,
       1.15651764e+00, 1.58197671e+00, 2.00000000e+00, 2.54149408e+00,
       5.00000000e+00, 5.51665557e+00, 1.00000000e+01, 2.00000000e+01,
       5.00000000e+01, 1.00000000e+02, 2.00000000e+02, 5.00000000e+02,
       1.00000000e+03, 2.00000000e+03, 5.00000000e+03])

In [26]:
ari = np.divide(1,ey)
ari

array([8.33333333e-02, 1.66666667e-01, 2.50000000e-01, 3.33333333e-01,
       5.00000000e-01, 1.00000000e+00, 1.44269504e+00, 2.00000000e+00,
       4.48142012e+00, 5.00000000e+00, 9.49122158e+00, 1.94957257e+01,
       4.94983165e+01, 9.94991625e+01, 1.99499582e+02, 4.99499833e+02,
       9.99499917e+02, 1.99949996e+03, 4.99949998e+03])

In [27]:
aep_percentage

array([9.99993856e+01, 9.97521248e+01, 9.81684361e+01, 9.50212932e+01,
       8.64664717e+01, 6.32120559e+01, 5.00000000e+01, 3.93469340e+01,
       2.00000000e+01, 1.81269247e+01, 1.00000000e+01, 5.00000000e+00,
       2.00000000e+00, 1.00000000e+00, 5.00000000e-01, 2.00000000e-01,
       1.00000000e-01, 5.00000000e-02, 2.00000000e-02])

In [28]:
frequency_window = np.array(object=['frequent', 'intermediate', 'rare'])[
    np.add(np.digitize(x=aep_percentage, bins=np.array(object=[100, 14.4, 3.2, 0]), right=False), np.negative(1))
    ]
frequency_window

array(['frequent', 'frequent', 'frequent', 'frequent', 'frequent',
       'frequent', 'frequent', 'frequent', 'frequent', 'frequent',
       'intermediate', 'intermediate', 'rare', 'rare', 'rare', 'rare',
       'rare', 'rare', 'rare'], dtype='<U12')

In [29]:
frequency_tag = np.concatenate((frequency_tag, np.array(object=['freq_1_in_5000'])))
frequency_tag

array(['freq_12EY', 'freq_6EY', 'freq_4EY', 'freq_3EY', 'freq_2EY',
       'freq_63_2_perc', 'freq_50_perc', 'freq_0_5EY', 'freq_20_perc',
       'freq_0_2EY', 'freq_10_perc', 'freq_5_perc', 'freq_2_perc',
       'freq_1_perc', 'freq_1_in_200', 'freq_1_in_500', 'freq_1_in_1000',
       'freq_1_in_2000', 'freq_1_in_5000'], dtype=object)

In [30]:
frequency_label = np.concatenate((frequency_label, np.array(object=['1 in 5000'])))
frequency_label

array(['12EY', '6EY', '4EY', '3EY', '2EY', '63.2%', '50%', '0.5EY', '20%',
       '0.2EY', '10%', '5%', '2%', '1%', '1 in 200', '1 in 500',
       '1 in 1000', '1 in 2000', '1 in 5000'], dtype=object)

In [31]:
frequency_table = pd.DataFrame(
    data=np.column_stack(tup=(ey, aep_percentage, aep_1_in_x, ari, frequency_window, frequency_label, frequency_tag)),
    columns=['EY', 'AEP_percentage', 'AEP_1_in_x', 'ARI', 'freq_window', 'freq_label', 'freq_tag']
    )
frequency_table.to_csv(path_or_buf='{}table_frequency.csv'.format(output_dir), index=False)
frequency_table.to_parquet(path='{}table_frequency.parquet'.format(output_dir))
frequency_table

,EY,AEP_percentage,...,freq_label,freq_tag
0,12.0,99.999386,...,12EY,freq_12EY
1,6.0,99.752125,...,6EY,freq_6EY
2,4.0,98.168436,...,4EY,freq_4EY
3,3.0,95.021293,...,3EY,freq_3EY
4,2.0,86.466472,...,2EY,freq_2EY
5,1.0,63.212056,...,63.2%,freq_63_2_perc
6,0.693147,50.0,...,50%,freq_50_perc
7,0.5,39.346934,...,0.5EY,freq_0_5EY
8,0.223144,20.0,...,20%,freq_20_perc
9,0.2,18.126925,...,0.2EY,freq_0_2EY


In [32]:
r_ey = ey.copy()
r_ey[:14] = r_ey[:14].round(decimals=2)
r_ey[14:17] = r_ey[14:17].round(decimals=3)
r_ey[17:] = r_ey[17:].round(decimals=4)
r_ey

array([1.2e+01, 6.0e+00, 4.0e+00, 3.0e+00, 2.0e+00, 1.0e+00, 6.9e-01,
       5.0e-01, 2.2e-01, 2.0e-01, 1.1e-01, 5.0e-02, 2.0e-02, 1.0e-02,
       5.0e-03, 2.0e-03, 1.0e-03, 5.0e-04, 2.0e-04])

In [33]:
r_aep_percentage = aep_percentage.copy()
r_aep_percentage = r_aep_percentage.round(decimals=2)
r_aep_percentage

array([1.000e+02, 9.975e+01, 9.817e+01, 9.502e+01, 8.647e+01, 6.321e+01,
       5.000e+01, 3.935e+01, 2.000e+01, 1.813e+01, 1.000e+01, 5.000e+00,
       2.000e+00, 1.000e+00, 5.000e-01, 2.000e-01, 1.000e-01, 5.000e-02,
       2.000e-02])

In [34]:
r_aep_1_in_x = aep_1_in_x.copy()
r_aep_1_in_x[:2] = r_aep_1_in_x[:2].round(decimals=3)
r_aep_1_in_x[2:] = r_aep_1_in_x[2:].round(decimals=2)
r_aep_1_in_x

array([1.000e+00, 1.002e+00, 1.020e+00, 1.050e+00, 1.160e+00, 1.580e+00,
       2.000e+00, 2.540e+00, 5.000e+00, 5.520e+00, 1.000e+01, 2.000e+01,
       5.000e+01, 1.000e+02, 2.000e+02, 5.000e+02, 1.000e+03, 2.000e+03,
       5.000e+03])

In [35]:
r_ari = ari.copy()
r_ari = r_ari.round(decimals=2)
r_ari

array([8.0000e-02, 1.7000e-01, 2.5000e-01, 3.3000e-01, 5.0000e-01,
       1.0000e+00, 1.4400e+00, 2.0000e+00, 4.4800e+00, 5.0000e+00,
       9.4900e+00, 1.9500e+01, 4.9500e+01, 9.9500e+01, 1.9950e+02,
       4.9950e+02, 9.9950e+02, 1.9995e+03, 4.9995e+03])

In [36]:
r_frequency_table = pd.DataFrame(
    data=np.column_stack(tup=(r_ey, r_aep_percentage, r_aep_1_in_x, r_ari, frequency_window, frequency_label, frequency_tag)),
    columns=['EY', 'AEP_percentage', 'AEP_1_in_x', 'ARI', 'freq_window', 'freq_label', 'freq_tag']
    )
r_frequency_table.to_csv(path_or_buf='{}table_rounded_frequency.csv'.format(output_dir), index=False)
r_frequency_table.to_parquet(path='{}table_rounded_frequency.parquet'.format(output_dir))
r_frequency_table

,EY,AEP_percentage,...,freq_label,freq_tag
0,12.0,100.0,...,12EY,freq_12EY
1,6.0,99.75,...,6EY,freq_6EY
2,4.0,98.17,...,4EY,freq_4EY
3,3.0,95.02,...,3EY,freq_3EY
4,2.0,86.47,...,2EY,freq_2EY
5,1.0,63.21,...,63.2%,freq_63_2_perc
6,0.69,50.0,...,50%,freq_50_perc
7,0.5,39.35,...,0.5EY,freq_0_5EY
8,0.22,20.0,...,20%,freq_20_perc
9,0.2,18.13,...,0.2EY,freq_0_2EY


# merge stats and pattern tables into timeseries

In [37]:
{key:val for key,val in enumerate(storm_stats_table.columns)}

{0: 'Event_ID',
 1: 'Region',
 2: 'Region_source',
 3: 'Burst_Duration_in_min',
 4: 'Burst_Loading',
 5: 'Original_Burst_Depth_in_mm',
 6: 'AEP_Window',
 7: 'AEP_source_in_perc',
 8: 'Burst_Start_Date',
 9: 'Burst_End_Date',
 10: 'DB_Event_Reference_Number',
 11: 'DB_Pluviograph_Reference_Number',
 12: 'Offical_Gauge',
 13: 'Lat',
 14: 'Long'}

In [38]:
df_storm_stats = storm_stats_table.copy()
df_storm_stats.drop(columns=storm_stats_table.columns[[1,2,4,5,7,8,9,10,11,12,13,14]], inplace=True)
df_storm_stats.rename(columns={'Event_ID':'event_id', 'Burst_Duration_in_min':'duration_in_min', 'AEP_Window':'freq_window'}, inplace=True)
df_storm_stats['id_duration'] = pd.Series(data=np.unique(ar=df_storm_stats.duration_in_min.to_numpy(), return_inverse=True)[1])
df_storm_stats['id_window'] = pd.Series(data=np.unique(ar=df_storm_stats.freq_window.to_numpy(), return_inverse=True)[1])
df_storm_stats['id_pattern'] = pd.Series(data=list(np.arange(stop=10))*int(len(df_storm_stats)/10))
df_storm_stats = df_storm_stats[df_storm_stats.columns[[0,4,3,5,2,1]]]

df_storm_stats

,event_id,id_window,...,freq_window,duration_in_min
0,4380,0,...,frequent,10
1,4382,0,...,frequent,10
2,4384,0,...,frequent,10
3,4385,0,...,frequent,10
4,4386,0,...,frequent,10
...,...,...,...,...,...
715,2856,2,...,rare,10080
716,5064,2,...,rare,10080
717,5069,2,...,rare,10080
718,5071,2,...,rare,10080


In [39]:
df_temp_pattern = temp_pattern_table.copy()
# df_temp_pattern['time_series'] = df_temp_pattern.apply(func=lambda arg: arg.to_numpy()[5:], axis=1)
df_temp_pattern['time_series'] = df_temp_pattern.apply(func=lambda arg: [ind for ind in arg.to_list()[5:] if str(ind) != 'nan'], axis=1)
df_temp_pattern.drop(columns=df_temp_pattern.columns[5:-1], inplace=True)
df_temp_pattern.drop(columns='Region', inplace=True)
df_temp_pattern.rename(columns={'EventID':'event_id', 'Duration':'duration_in_min', 'TimeStep':'time_step', 'AEP':'freq_window'}, inplace=True)
df_temp_pattern['id_duration'] = pd.Series(data=np.unique(ar=df_temp_pattern.duration_in_min.to_numpy(), return_inverse=True)[1])
df_temp_pattern['id_timestep'] = pd.Series(data=np.unique(ar=df_temp_pattern.time_step.to_numpy(), return_inverse=True)[1])
df_temp_pattern['id_window'] = pd.Series(data=np.unique(ar=df_temp_pattern.freq_window.to_numpy(), return_inverse=True)[1])
df_temp_pattern['id_pattern'] = pd.Series(data=list(np.arange(stop=10))*int(len(df_temp_pattern)/10))
df_temp_pattern = df_temp_pattern[df_temp_pattern.columns[[0,7,5,6,8,3,1,2,4]]]

df_temp_pattern

,event_id,id_window,...,time_step,time_series
0,4380,0,...,5,"[58.06, 41.94]"
1,4382,0,...,5,"[52.13, 47.87]"
2,4384,0,...,5,"[60.71, 39.29]"
3,4385,0,...,5,"[51.51, 48.49]"
4,4386,0,...,5,"[54.55, 45.45]"
...,...,...,...,...,...
715,2856,2,...,180,"[1.14, 1.8, 1.36, 1.29, 3.2, 4.78, 3.28, 3.13, 2.57, 5.08, 4.1, 3.35, 5.76, 0.78, 0.59, 3.04, 0.32, 0.01, 0.0, 0.89, 4.98, 6.96, 1.24, 0.71, 1.3, 2.23, 6.88, 3.58, 3.67, 3.89, 4.15, 3.05, 1.15, 0...."
716,5064,2,...,180,"[0.61, 0.93, 0.26, 0.04, 2.91, 1.17, 4.05, 0.16, 0.04, 0.05, 0.13, 0.23, 4.97, 0.45, 2.51, 6.24, 0.76, 2.16, 2.7, 3.1, 5.7, 6.88, 2.52, 1.74, 5.39, 3.0, 0.72, 1.38, 3.4, 4.6, 4.98, 1.87, 0.87, 1.0..."
717,5069,2,...,180,"[0.0, 0.0, 0.0, 1.06, 0.04, 0.41, 0.71, 0.01, 1.13, 4.69, 2.46, 10.14, 0.0, 0.05, 0.05, 0.02, 1.19, 2.08, 2.22, 0.34, 3.01, 0.68, 0.1, 2.73, 3.12, 1.95, 0.54, 0.62, 0.49, 0.38, 2.31, 5.12, 15.82, ..."
718,5071,2,...,180,"[1.21, 0.0, 0.0, 0.0, 0.0, 0.04, 0.52, 2.96, 1.0, 0.0, 0.05, 4.24, 1.54, 7.54, 2.53, 6.91, 6.07, 3.6, 3.97, 0.05, 0.11, 0.74, 0.24, 0.81, 1.8, 4.78, 1.6, 0.81, 1.4, 0.0, 2.05, 2.26, 5.77, 0.0, 0.4..."


In [40]:
{key:val for key,val in enumerate(df_temp_pattern)}

{0: 'event_id',
 1: 'id_window',
 2: 'id_duration',
 3: 'id_timestep',
 4: 'id_pattern',
 5: 'freq_window',
 6: 'duration_in_min',
 7: 'time_step',
 8: 'time_series'}

In [41]:
df_timeseries = pd.merge(left=df_storm_stats, right=df_temp_pattern, how='inner', on='event_id')# left_on='Event_ID', right_on='EventID')
df_timeseries.drop(columns=df_timeseries.columns[[0,6,7,9,10,11]], inplace=True)
df_timeseries.rename(columns=df_timeseries.columns[:5].to_series().str.rsplit(pat='_', n=1).str[0].to_dict(), inplace=True)
df_timeseries = df_timeseries[df_timeseries.columns[[0,1,5,2,3,4,6,7]]]

df_timeseries

,id_window,id_duration,...,time_step,time_series
0,0,0,...,5,"[58.06, 41.94]"
1,0,0,...,5,"[52.13, 47.87]"
2,0,0,...,5,"[60.71, 39.29]"
3,0,0,...,5,"[51.51, 48.49]"
4,0,0,...,5,"[54.55, 45.45]"
...,...,...,...,...,...
715,2,23,...,180,"[1.14, 1.8, 1.36, 1.29, 3.2, 4.78, 3.28, 3.13, 2.57, 5.08, 4.1, 3.35, 5.76, 0.78, 0.59, 3.04, 0.32, 0.01, 0.0, 0.89, 4.98, 6.96, 1.24, 0.71, 1.3, 2.23, 6.88, 3.58, 3.67, 3.89, 4.15, 3.05, 1.15, 0...."
716,2,23,...,180,"[0.61, 0.93, 0.26, 0.04, 2.91, 1.17, 4.05, 0.16, 0.04, 0.05, 0.13, 0.23, 4.97, 0.45, 2.51, 6.24, 0.76, 2.16, 2.7, 3.1, 5.7, 6.88, 2.52, 1.74, 5.39, 3.0, 0.72, 1.38, 3.4, 4.6, 4.98, 1.87, 0.87, 1.0..."
717,2,23,...,180,"[0.0, 0.0, 0.0, 1.06, 0.04, 0.41, 0.71, 0.01, 1.13, 4.69, 2.46, 10.14, 0.0, 0.05, 0.05, 0.02, 1.19, 2.08, 2.22, 0.34, 3.01, 0.68, 0.1, 2.73, 3.12, 1.95, 0.54, 0.62, 0.49, 0.38, 2.31, 5.12, 15.82, ..."
718,2,23,...,180,"[1.21, 0.0, 0.0, 0.0, 0.0, 0.04, 0.52, 2.96, 1.0, 0.0, 0.05, 4.24, 1.54, 7.54, 2.53, 6.91, 6.07, 3.6, 3.97, 0.05, 0.11, 0.74, 0.24, 0.81, 1.8, 4.78, 1.6, 0.81, 1.4, 0.0, 2.05, 2.26, 5.77, 0.0, 0.4..."


In [42]:
{key:val for key,val in enumerate(df_timeseries)}

{0: 'id_window',
 1: 'id_duration',
 2: 'id_timestep',
 3: 'id_pattern',
 4: 'freq_window',
 5: 'duration_in_min',
 6: 'time_step',
 7: 'time_series'}

# idf depths

In [43]:
df_timeseries.columns

Index(['id_window', 'id_duration', 'id_timestep', 'id_pattern', 'freq_window',
       'duration_in_min', 'time_step', 'time_series'],
      dtype='object')

In [44]:
df_map = df_timeseries.copy()
df_map = df_map[['duration_in_min', 'time_step']]#.iloc[:,:2].drop_duplicates(subset=['time_step', 'duration_in_min'])
df_map.reset_index(drop=True, inplace=True)
df_map = {key:val for key,val in zip(df_map.duration_in_min, df_map.time_step)}

df_map

{10: 5,
 15: 5,
 20: 5,
 25: 5,
 30: 5,
 45: 5,
 60: 5,
 90: 5,
 120: 5,
 180: 15,
 270: 15,
 360: 15,
 540: 30,
 720: 30,
 1080: 60,
 1440: 60,
 1800: 120,
 2160: 120,
 2880: 120,
 4320: 180,
 5760: 180,
 7200: 180,
 8640: 180,
 10080: 180}

In [45]:
df_id = frequency_table.copy()
df_id.drop(index=len(df_id)-1, inplace=True)
df_id = df_id[['freq_window', 'freq_label', 'freq_tag']]#.iloc[:-1,4:]
df_id = pd.concat(objs=[
    pd.Series(data=np.full(shape=len(df_id), fill_value='depth')),
    df_id,
    pd.Series(data=np.unique(ar=df_id.freq_window.to_numpy(), return_inverse=True)[1], name='id_window'),
    df_id.index.to_series(name='id_tag')
    ], axis=1)
df_id = df_id[df_id.columns[[0,4,5,1,2,3]]]

df_id

,0,id_window,...,freq_label,freq_tag
0,depth,0,...,12EY,freq_12EY
1,depth,0,...,6EY,freq_6EY
2,depth,0,...,4EY,freq_4EY
3,depth,0,...,3EY,freq_3EY
4,depth,0,...,2EY,freq_2EY
5,depth,0,...,63.2%,freq_63_2_perc
6,depth,0,...,50%,freq_50_perc
7,depth,0,...,0.5EY,freq_0_5EY
8,depth,0,...,20%,freq_20_perc
9,depth,0,...,0.2EY,freq_0_2EY


In [46]:
df_idf = idf_table.copy()
df_idf = df_idf[df_idf.Duration_in_min >= 10]
df_idf.reset_index(drop=True, inplace=True)
df_idf.rename(columns={'Duration':'duration', 'Duration_in_min':'duration_in_min'}, inplace=True)
df_idf.duration_in_min = df_idf.duration_in_min.astype(int)
df_idf['id_duration'] = pd.Series(data=np.unique(ar=df_idf.duration_in_min.to_numpy(), return_inverse=True)[1])
df_idf['time_step'] = df_idf.duration_in_min.map(arg=df_map)
df_idf['id_timestep'] = pd.Series(data=np.unique(ar=df_idf.time_step.to_numpy(), return_inverse=True)[1])
df_idf = df_idf[df_idf.columns[-3:].to_list() + df_idf.columns[:-3].to_list()]
df_idf.set_index(keys=df_idf.columns[:6].to_list(), inplace=True)
df_idf.columns = pd.MultiIndex.from_frame(df=df_id)
df_idf = df_idf.stack(level=[1,2,3,4,5], future_stack=True)
df_idf.reset_index(inplace=True)
df_idf = df_idf[df_idf.columns[[6,0,2,7,3,4,8,5,1,9,10,11]]]

df_idf

,id_window,id_duration,...,freq_tag,depth
0,0,0,...,freq_12EY,5.15
1,0,0,...,freq_6EY,5.91
2,0,0,...,freq_4EY,7.28
3,0,0,...,freq_3EY,8.27
4,0,0,...,freq_2EY,9.70
...,...,...,...,...,...
427,2,23,...,freq_1_perc,462.00
428,2,23,...,freq_1_in_200,514.00
429,2,23,...,freq_1_in_500,578.00
430,2,23,...,freq_1_in_1000,628.00


# rainfall_patterns

In [47]:
rain_data = pd.merge(left=df_timeseries, right=df_idf, how='cross', suffixes=[None, '_'])
rain_data = rain_data[
    (rain_data.id_window == rain_data.id_window_) & 
    (rain_data.id_duration == rain_data.id_duration_) & 
    (rain_data.id_timestep == rain_data.id_timestep_)
    ]
rain_data.drop(columns=rain_data.columns[[8,9,10,14,15,16]], inplace=True)
rain_data = rain_data[rain_data.columns[[0,1,2,8,3,9,10,4,5,6,11,12,13,7]]]
rain_data.drop_duplicates(subset=rain_data.columns[:-1], inplace=True)
# rain_data.insert()
rain_data.sort_values(by=['id_tag', 'id_timestep', 'id_duration', 'id_pattern'], inplace=True)
# rain_data.sort_values(by=['id_pattern'], inplace=True)
rain_data.reset_index(drop=True, inplace=True)
rain_data['label'] = (
    (rain_data.index.astype(str).str.len().max() - rain_data.index.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.index)) + rain_data.index.astype(str) + '__' +
    (rain_data.id_tag.astype(str).str.len().max() - rain_data.id_tag.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_tag)) + rain_data.id_tag.astype(str) + '_' +
    (rain_data.id_timestep.astype(str).str.len().max() - rain_data.id_timestep.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_timestep)) + rain_data.id_timestep.astype(str) + '_' +
    (rain_data.id_duration.astype(str).str.len().max() - rain_data.id_duration.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_duration)) + rain_data.id_duration.astype(str) + '_' +
    (rain_data.id_pattern.astype(str).str.len().max() - rain_data.id_pattern.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_pattern)) + rain_data.id_pattern.astype(str) + '_' +
    (rain_data.id_window.astype(str).str.len().max() - rain_data.id_window.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_window)) + rain_data.id_window.astype(str) + '__' +
    (rain_data.freq_window.str.len().max() - rain_data.freq_window.str.len())*pd.Series(data=['_']*len(rain_data.freq_window)) + rain_data.freq_window + '__' +
    (rain_data.freq_tag.str.len().max() - rain_data.freq_tag.str.len())*pd.Series(data=['_']*len(rain_data.freq_tag)) + rain_data.freq_tag + '__' +
    (rain_data.time_step.astype(str).str.len().max() - rain_data.time_step.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.time_step)) + rain_data.time_step.astype(str) + '__' +
    (rain_data.duration_in_min.astype(str).str.len().max() - rain_data.duration_in_min.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.duration_in_min)) + rain_data.duration_in_min.astype(str) + '__' +
    (rain_data.id_pattern.astype(str).str.len().max() - rain_data.id_pattern.astype(str).str.len())*pd.Series(data=['0']*len(rain_data.id_pattern)) + rain_data.id_pattern.astype(str)
    )
rain_data = rain_data[rain_data.columns[:8].to_list() + rain_data.columns[-1:].to_list() + rain_data.columns[8:-1].to_list()]
rain_data.drop(columns=rain_data.columns[:5], inplace=True)

rain_data

,duration,units,...,depth,time_series
0,10.0,min,...,5.15,"[58.06, 41.94]"
1,10.0,min,...,5.15,"[52.13, 47.87]"
2,10.0,min,...,5.15,"[60.71, 39.29]"
3,10.0,min,...,5.15,"[51.51, 48.49]"
4,10.0,min,...,5.15,"[54.55, 45.45]"
...,...,...,...,...,...
4315,168.0,hour,...,677.00,"[1.14, 1.8, 1.36, 1.29, 3.2, 4.78, 3.28, 3.13, 2.57, 5.08, 4.1, 3.35, 5.76, 0.78, 0.59, 3.04, 0.32, 0.01, 0.0, 0.89, 4.98, 6.96, 1.24, 0.71, 1.3, 2.23, 6.88, 3.58, 3.67, 3.89, 4.15, 3.05, 1.15, 0...."
4316,168.0,hour,...,677.00,"[0.61, 0.93, 0.26, 0.04, 2.91, 1.17, 4.05, 0.16, 0.04, 0.05, 0.13, 0.23, 4.97, 0.45, 2.51, 6.24, 0.76, 2.16, 2.7, 3.1, 5.7, 6.88, 2.52, 1.74, 5.39, 3.0, 0.72, 1.38, 3.4, 4.6, 4.98, 1.87, 0.87, 1.0..."
4317,168.0,hour,...,677.00,"[0.0, 0.0, 0.0, 1.06, 0.04, 0.41, 0.71, 0.01, 1.13, 4.69, 2.46, 10.14, 0.0, 0.05, 0.05, 0.02, 1.19, 2.08, 2.22, 0.34, 3.01, 0.68, 0.1, 2.73, 3.12, 1.95, 0.54, 0.62, 0.49, 0.38, 2.31, 5.12, 15.82, ..."
4318,168.0,hour,...,677.00,"[1.21, 0.0, 0.0, 0.0, 0.0, 0.04, 0.52, 2.96, 1.0, 0.0, 0.05, 4.24, 1.54, 7.54, 2.53, 6.91, 6.07, 3.6, 3.97, 0.05, 0.11, 0.74, 0.24, 0.81, 1.8, 4.78, 1.6, 0.81, 1.4, 0.0, 2.05, 2.26, 5.77, 0.0, 0.4..."


In [48]:
def build_df_timeseries(arg_time_series, arg_depth, arg_time_step, arg_sim_starting_time, arg_timeseries_name):

    rain_value = pd.Series(data=np.concatenate((
        np.array(object=[0]),
        np.divide(np.multiply(arg_depth, np.array(object=arg_time_series)), 100),
        np.array(object=[0])
        )), name='Value')

    rain_date = pd.Series(data=pd.date_range(
        start=arg_sim_starting_time, 
        periods=rain_value.size,
        freq=pd.Timedelta(minutes=arg_time_step)
        ), name='Date').dt.strftime(date_format='%m/%d/%Y')

    rain_time = pd.Series(data=pd.date_range(
        start=arg_sim_starting_time, 
        periods=rain_value.size,
        freq=pd.Timedelta(minutes=arg_time_step)
        ), name='Time').dt.strftime(date_format='%H:%M')
    
    rain_gauge_timeseries = pd.Series(data=[arg_timeseries_name]*rain_value.size, name='Name')

    df_rain_series = pd.concat(objs=[rain_gauge_timeseries, rain_date, rain_time, rain_value], axis=1)
    df_rain_series.set_index(keys='Name', inplace=True)

    return df_rain_series

In [49]:
rain_data.time_series = rain_data.apply(func=lambda arg: build_df_timeseries(arg.time_series, arg.depth, arg.time_step, simulation_starting_time, timeseries_name), axis=1)
rain_data

,duration,units,...,depth,time_series
0,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.00000 RainGauge 01/01/2000 00:05 2.99009 RainGauge 01/01/2000 00:10 2.15991 RainGa...
1,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.000000 RainGauge 01/01/2000 00:05 2.684695 RainGauge 01/01/2000 00:10 2.465305 R...
2,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.000000 RainGauge 01/01/2000 00:05 3.126565 RainGauge 01/01/2000 00:10 2.023435 R...
3,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.000000 RainGauge 01/01/2000 00:05 2.652765 RainGauge 01/01/2000 00:10 2.497235 R...
4,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.000000 RainGauge 01/01/2000 00:05 2.809325 RainGauge 01/01/2000 00:10 2.340675 R...
...,...,...,...,...,...
4315,168.0,hour,...,677.00,Date Time Value Name RainGauge 01/01/2000 00:00 0.0000 RainGauge 01/01/2000 03:00 7.7178 RainGauge 01/01/2000 06:00 12.1860 RainGa...
4316,168.0,hour,...,677.00,Date Time Value Name RainGauge 01/01/2000 00:00 0.0000 RainGauge 01/01/2000 03:00 4.1297 RainGauge 01/01/2000 06:00 6.2961 RainGa...
4317,168.0,hour,...,677.00,Date Time Value Name RainGauge 01/01/2000 00:00 0.0000 RainGauge 01/01/2000 03:00 0.0000 RainGauge 01/01/2000 06:00 0.0000 R...
4318,168.0,hour,...,677.00,Date Time Value Name RainGauge 01/01/2000 00:00 0.0000 RainGauge 01/01/2000 03:00 8.1917 RainGauge 01/01/2000 06:00 0.0000 RainGa...


In [50]:
rain_data = rain_data.copy().iloc[:120]
rain_data

,duration,units,...,depth,time_series
0,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.00000 RainGauge 01/01/2000 00:05 2.99009 RainGauge 01/01/2000 00:10 2.15991 RainGa...
1,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.000000 RainGauge 01/01/2000 00:05 2.684695 RainGauge 01/01/2000 00:10 2.465305 R...
2,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.000000 RainGauge 01/01/2000 00:05 3.126565 RainGauge 01/01/2000 00:10 2.023435 R...
3,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.000000 RainGauge 01/01/2000 00:05 2.652765 RainGauge 01/01/2000 00:10 2.497235 R...
4,10.0,min,...,5.15,Date Time Value Name RainGauge 01/01/2000 00:00 0.000000 RainGauge 01/01/2000 00:05 2.809325 RainGauge 01/01/2000 00:10 2.340675 R...
...,...,...,...,...,...
115,6.0,hour,...,20.40,Date Time Value Name RainGauge 01/01/2000 00:00 0.00000 RainGauge 01/01/2000 00:15 2.60100 RainGauge 01/01/2000 00:30 1.97676 RainGa...
116,6.0,hour,...,20.40,Date Time Value Name RainGauge 01/01/2000 00:00 0.00000 RainGauge 01/01/2000 00:15 0.62016 RainGauge 01/01/2000 00:30 0.64260 RainGa...
117,6.0,hour,...,20.40,Date Time Value Name RainGauge 01/01/2000 00:00 0.00000 RainGauge 01/01/2000 00:15 2.15424 RainGauge 01/01/2000 00:30 1.19748 RainGa...
118,6.0,hour,...,20.40,Date Time Value Name RainGauge 01/01/2000 00:00 0.00000 RainGauge 01/01/2000 00:15 0.60180 RainGauge 01/01/2000 00:30 0.48144 RainGa...


In [51]:
def create_inp_files(arg_label, arg_time_series, arg_time_step, arg_inp_file_dir, arg_inp_file_path, arg_rain_stopping_time, arg_timeseries_name):

    inp_file = '{}inp_file__{}.inp'.format(arg_inp_file_dir, arg_label)

    shutil.copyfile(src=arg_inp_file_path, dst=inp_file)

    end_date = pd.to_datetime(arg='{} {}'.format(arg_time_series.Date.iloc[-1], arg_time_series.Time.iloc[-1]), format='%m/%d/%Y %H:%M')
    end_date = end_date + arg_rain_stopping_time
    end_date, end_time = end_date.strftime(format='%m/%d/%Y'), end_date.strftime(format='%H:%M:%S')

    options_section = swmmio.utils.dataframes.dataframe_from_inp(inp_path=inp_file, section='[OPTIONS]')
    options_section.at['END_DATE', 'Value'] = end_date
    options_section.at['END_TIME', 'Value'] = end_time

    time_interval = str(pd.Timedelta(minutes=arg_time_step)).split(sep=' ')[-1].rsplit(sep=':', maxsplit=1)[0]

    raingauges_section = swmmio.utils.dataframes.dataframe_from_inp(inp_path=inp_file, section='[RAINGAGES]')
    raingauges_section.at[arg_timeseries_name, 'TimeIntrvl'] = time_interval

    [swmmio.utils.modify_model.replace_inp_section(inp_path=inp_file, modified_section_header=ind1, new_data=ind2)
    for ind1, ind2 in zip(
        ['[OPTIONS]', '[RAINGAGES]', '[TIMESERIES]'], 
        [options_section, raingauges_section, arg_time_series]
        )]

In [52]:
rain_data.apply(func=lambda arg: create_inp_files(arg.label, arg.time_series, arg.time_step, inp_file_dir, inp_file_path, stopping_time_after_precipitation_finish, timeseries_name),axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
115    None
116    None
117    None
118    None
119    None
Length: 120, dtype: object

In [53]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [54]:
for ind1 in rain_data.index:
    model = '{}inp_file__{}.inp'.format(inp_file_dir, rain_data.label[ind1])
    with Simulation(inputfile=model) as sim:
        link_sim = Links(model=sim)['116']
        time_stamp = []
        flow_rate = []
        for ind2, step in enumerate(sim):
            time_stamp.append(sim.current_time)
            flow_rate.append(link_sim.flow)
        
        time_stamp = pd.Series(data=time_stamp, name='time_stamp')
        flow_rate = pd.Series(data=flow_rate, name='flow_rate')
        flow_rate_sim = pd.concat(objs=[time_stamp, flow_rate], axis=1)
        # flow_rate_sim.to_parquet(path='{}flow_rate__{}.parquet'.format(flow_rate_dir, rain_data.label[ind1]))

In [55]:
flow_rate_sim

,time_stamp,flow_rate
0,2000-01-01 00:05:00,0.000000
1,2000-01-01 00:10:00,0.000000
2,2000-01-01 00:15:00,0.000000
3,2000-01-01 00:20:00,0.000000
4,2000-01-01 00:25:00,0.000002
...,...,...
141,2000-01-01 11:50:00,0.000171
142,2000-01-01 11:55:00,0.000164
143,2000-01-01 12:00:00,0.000148
144,2000-01-01 12:05:00,0.000134


In [56]:
flow_rate_sim.dtypes

time_stamp    datetime64[ns]
flow_rate            float64
dtype: object

In [ ]:
break

In [ ]:
timeseries_name

'RainGauge'

In [ ]:
rain_data.columns

Index(['id_window', 'id_duration', 'id_timestep', 'id_tag', 'id_pattern',
       'duration', 'units', 'freq_window', 'label', 'duration_in_min',
       'time_step', 'freq_label', 'freq_tag', 'depth', 'time_series'],
      dtype='object')

In [ ]:
inp_file_dir

'./Assignment_3__Output_FE__1st__centroid/inp_file_dir/'

In [ ]:
inp_file_path

'./Assignment_3__Output_FE__1st__centroid/inp_template.inp'

In [ ]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
inp_file_path

'./Assignment_3__Output_FE__1st__centroid/inp_template.inp'

In [ ]:
indd = 3000
indd

3000

In [ ]:
rain_data.time_series[indd]

,Date,Time,Value
Name,,,
RainGauge,01/01/2000,00:00,0.0000
RainGauge,01/01/2000,00:30,7.9524
RainGauge,01/01/2000,01:00,5.2875
RainGauge,01/01/2000,01:30,6.0630
RainGauge,01/01/2000,02:00,11.7453
RainGauge,01/01/2000,02:30,12.1260
RainGauge,01/01/2000,03:00,4.5543
RainGauge,01/01/2000,03:30,4.1736
RainGauge,01/01/2000,04:00,4.1736


In [ ]:
inp_file = '{}inp_file__{}.inp'.format(inp_file_dir, rain_data.label[indd])
inp_file

'./Assignment_3__Output_FE__1st__centroid/inp_file_dir/inp_file__3000__12_2_12_0_2__________rare_____freq_2_perc__030__00540__0.inp'

In [ ]:
shutil.copyfile(src=inp_file_path, dst=inp_file)

'./Assignment_3__Output_FE__1st__centroid/inp_file_dir/inp_file__3000__12_2_12_0_2__________rare_____freq_2_perc__030__00540__0.inp'

In [ ]:
end_date = pd.to_datetime(arg='{} {}'.format(rain_data.time_series[indd].Date.iloc[-1], rain_data.time_series[indd].Time.iloc[-1]), format='%m/%d/%Y %H:%M')
end_date = end_date + stopping_time_after_precipitation_finish
end_date, end_time = end_date.strftime(format='%m/%d/%Y'), end_date.strftime(format='%H:%M:%S')

end_date, end_time

('01/01/2000', '15:30:00')

In [ ]:
# reporting_step = str(pd.Timedelta(minutes=rain_data.time_step[indd])).split(sep=' ')[-1]
# reporting_step

In [ ]:
options_section = swmmio.utils.dataframes.dataframe_from_inp(inp_path=inp_file, section='[OPTIONS]')
options_section.at['END_DATE', 'Value'] = end_date
options_section.at['END_TIME', 'Value'] = end_time
# options_section.at['REPORT_STEP', 'Value'] = reporting_step
# options_section.at['WET_STEP', 'Value'] = reporting_step
# options_section.at['DRY_STEP', 'Value'] = reporting_step

options_section

,Value
Key,
FLOW_UNITS,CMS
INFILTRATION,HORTON
FLOW_ROUTING,KINWAVE
LINK_OFFSETS,DEPTH
MIN_SLOPE,0
ALLOW_PONDING,NO
SKIP_STEADY_STATE,NO
START_DATE,01/01/2000
START_TIME,00:00:00


In [ ]:
time_interval = str(pd.Timedelta(minutes=rain_data.time_step[indd])).split(sep=' ')[-1].rsplit(sep=':', maxsplit=1)[0]
time_interval

'00:30'

In [ ]:
raingauges_section = swmmio.utils.dataframes.dataframe_from_inp(inp_path=inp_file, section='[RAINGAGES]')
raingauges_section.at[timeseries_name, 'TimeIntrvl'] = time_interval

raingauges_section

,RainType,TimeIntrvl,SnowCatch,DataSource,DataSourceName
Name,,,,,
RainGauge,VOLUME,00:30,1.0,TIMESERIES,RainGauge


In [ ]:
timeseries_section = rain_data.time_series[indd]
timeseries_section

,Date,Time,Value
Name,,,
RainGauge,01/01/2000,00:00,0.0000
RainGauge,01/01/2000,00:30,7.9524
RainGauge,01/01/2000,01:00,5.2875
RainGauge,01/01/2000,01:30,6.0630
RainGauge,01/01/2000,02:00,11.7453
RainGauge,01/01/2000,02:30,12.1260
RainGauge,01/01/2000,03:00,4.5543
RainGauge,01/01/2000,03:30,4.1736
RainGauge,01/01/2000,04:00,4.1736


In [ ]:
[swmmio.utils.modify_model.replace_inp_section(inp_path=inp_file, modified_section_header=ind1, new_data=ind2)
for ind1, ind2 in zip(
    ['[OPTIONS]', '[RAINGAGES]', '[TIMESERIES]'], 
    [options_section, raingauges_section, timeseries_section]
    )]

In [ ]:
break

In [ ]:
rain_data.time_series[indd]

,Date,Time,Value
Name,,,
RainGauge,01/01/2000,00:00,0.0000
RainGauge,01/01/2000,00:30,7.9524
RainGauge,01/01/2000,01:00,5.2875
RainGauge,01/01/2000,01:30,6.0630
RainGauge,01/01/2000,02:00,11.7453
RainGauge,01/01/2000,02:30,12.1260
RainGauge,01/01/2000,03:00,4.5543
RainGauge,01/01/2000,03:30,4.1736
RainGauge,01/01/2000,04:00,4.1736


In [ ]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
rain_data.columns

Index(['id_window', 'id_duration', 'id_timestep', 'id_tag', 'id_pattern',
       'duration', 'units', 'freq_window', 'label', 'duration_in_min',
       'time_step', 'freq_label', 'freq_tag', 'depth', 'time_series'],
      dtype='object')

In [ ]:
a = 0
b = a+60
rain_data.iloc[a:b]

,id_window,id_duration,id_timestep,id_tag,id_pattern,duration,units,freq_window,label,duration_in_min,time_step,freq_label,freq_tag,depth,time_series
0,0,0,0,0,0,10.0,min,frequent,rain__0000__00_0_00_0_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
1,0,0,0,0,1,10.0,min,frequent,rain__0001__00_0_00_1_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
2,0,0,0,0,2,10.0,min,frequent,rain__0002__00_0_00_2_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
3,0,0,0,0,3,10.0,min,frequent,rain__0003__00_0_00_3_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
4,0,0,0,0,4,10.0,min,frequent,rain__0004__00_0_00_4_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
5,0,0,0,0,5,10.0,min,frequent,rain__0005__00_0_00_5_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
6,0,0,0,0,6,10.0,min,frequent,rain__0006__00_0_00_6_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
7,0,0,0,0,7,10.0,min,frequent,rain__0007__00_0_00_7_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
8,0,0,0,0,8,10.0,min,frequent,rain__0008__00_0_00_8_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...
9,0,0,0,0,9,10.0,min,frequent,rain__0009__00_0_00_9_0______frequent_______fr...,10,5,12EY,freq_12EY,5.15,Date Time Value Name ...


In [ ]:
indd = 50
indd = indd*18
rain_data.time_series[indd]

,Date,Time,Value
Name,,,
RainGauge,01/01/2000,00:00,0.00000
RainGauge,01/01/2000,02:00,1.75168
RainGauge,01/01/2000,04:00,1.47936
RainGauge,01/01/2000,06:00,1.25856
RainGauge,01/01/2000,08:00,1.96512
RainGauge,01/01/2000,10:00,4.87968
RainGauge,01/01/2000,12:00,4.02592
RainGauge,01/01/2000,14:00,0.58880
RainGauge,01/01/2000,16:00,0.87584


In [ ]:
rain_data.time_series[indd].Value.sum()

73.6

In [ ]:
pd.reset_option('^display.', silent=True)

In [ ]:
orig_name = '{}asdasd.out'.format(output_dir)
file_name = '{}qweqweqwe.inp'.format(output_dir)

print(orig_name, file_name, sep='\n')
shutil.copyfile(src=orig_name, dst=file_name)

./Assignment_3__Output_FE__1st__centroid/asdasd.out
./Assignment_3__Output_FE__1st__centroid/qweqweqwe.inp


'./Assignment_3__Output_FE__1st__centroid/qweqweqwe.inp'

In [ ]:
for ind in rain_timeseries.index:
    with open(file=file_name, mode='r+') as file:
        writer = file.readlines()
        writer.insert(0+ind, '{}\t{}\t{}\n'.format(rain_timeseries.rain_date[ind], rain_timeseries.rain_time[ind], rain_timeseries.rain_value[ind]))
        file.seek(0)
        file.writelines(writer)